In [4]:
import numpy as np 
import matplotlib.pyplot as plt 
import random

#creating initial random configuration
c = -1       #Plaquette coupling constant, favours an even number of alligned spins
b = 1.5      #Inverse temperature
n = 5        #Side length of lattice
N = n**2     #Number of sites in lattice
s = np.zeros([n,n])


for i in range(0,n):
    for j in range(0,n):
        a = random.randint(0,1)  #randomly assigns spin up or down with 50/50 chance for each
        if a == 0:
            s[i, j] = -1
        else:
            s[i, j] = 1
      
    
print (s)   #Shows inital random configuration

#calcualting energy of initial configuration 
E = 0
for i in range(0,n-1):
        for j in range(0,n-1):
            p = c * s[i,j] * s[i+1,j] * s[i, j+1] * s[i + 1, j + 1]
            E = E + p


#upper right plaquette interactions
def UR_plaquette_product(i, j, s, n):
    if (i == n-1) or (j == n - 1):
        r = 0                            #prevents issue of calculating energy at boundary lattice points-
    else:                                #- where there is no upper right plaquette
        r = s[i,j] * s[i+1,j] * s[i, j+1] * s[i + 1, j + 1]
        
    return r


snew = s.copy()   
sweep = 600010  #number of sweeps and therefore data points collected  
trans = 10      #number of configs 'ignored' before data starts being recorded



mcs = 0 
MCdata = np.zeros([sweep -trans,n,n])  #array for recording configurations (data points)
Edata = np.zeros([sweep-trans])        #array to record energy of each data point
while mcs < sweep:
    counter = mcs %100000   #keeps track of number of sweeps performed
    if counter == 0:
        print(mcs)
    
    #one MC sweep 
    z = 0
    while z <= N:
        
        #randomly selecting element and flipping its value
        k = random.randint(0,n-1)
        l = random.randint(0,n-1)
        snew[k,l] = -snew[k,l]
        
        #working out change in energy when flipping value on a site 
        dE = 0
        for i in range(max(k-1,0),k+1):
            for j in range(max(l-1,0),l+1):
                p =  UR_plaquette_product(i, j, snew, n)
                dE = dE - (2*p)
                
                #deciding whether to keep new or old configuration
        r = random.random()
        if dE < 0:
            snew[k,l] = snew[k,l] 
        else:
            if r < np.exp(-b*dE):
                snew[k,l] = snew[k,l] 
            else:  
                snew[k,l] = -snew[k,l] #goes back to old config if energy increases and the Boltzmann factor-  
        z += 1                         #-is too small i.e increase in energy is too large
        
        #calculating energy for configuration so it can be recorded to energy array
    Enew = 0
    for i in range(0,n-1):
        for j in range(0,n-1):
            p = c * snew[i,j] * snew[i+1,j] * snew[i, j+1] * snew[i + 1, j + 1]
            Enew = Enew + p
                
    
   
    
    #only starts to record data after transient of 10 mc sweeps 
    if trans <= mcs:
        Edata[mcs-trans] = Enew   #helps indicate how much the energy of configs had decreased from the-
        MCdata[mcs-trans] = snew  #- initial random one  
    mcs += 1
    
#np.savetxt('Edata.txt',Edata)
np.save('MCdata(5x5)(600,000).npy', MCdata)

print(snew)   #gives visusal insight into how the structure of the configurations has changed after
              #- MC algorithm i.e more groups of spins alligned 
            

[[-1.  1.  1.  1. -1.]
 [ 1.  1.  1. -1.  1.]
 [-1. -1. -1.  1. -1.]
 [ 1. -1. -1.  1.  1.]
 [-1.  1. -1. -1.  1.]]
-2.0
0
100000
200000
300000
400000
500000
600000
[[-1. -1.  1. -1.  1.]
 [-1. -1.  1. -1.  1.]
 [-1. -1.  1. -1.  1.]
 [-1. -1.  1. -1.  1.]
 [-1. -1.  1. -1.  1.]]


# Calculating the average values for each S_i so that it can be used in gradient function calculation

In [7]:


#Done here rather than within the learning algorithm- 
#-to reduce number of actions performed in learning, so runtime can be minimised

samples = len(MCdata)  #gives number of data points recorded 

S_i_avg_Data = np.zeros([n,n])

for i in range(0,samples):
    for j in range(0,n):
        for k in range(0,n):
            S_i_avg_Data[j,k] = S_i_avg_Data[j,k] + (MCdata[i,j,k]/samples) 
            
print(S_i_avg_Data)
np.save('S_i_avg_Data(5x5)(600,000).npy', S_i_avg_Data) #saves data so it can be loaded in for learning algotithm 



600000
[[ 0.01459333  0.01648     0.00357667 -0.00707667  0.01276667]
 [ 0.03139667 -0.00972333 -0.07705333  0.09010667 -0.02655667]
 [-0.00349    -0.00228     0.13484667  0.09029333  0.00214667]
 [-0.00122     0.17252667 -0.02034667  0.19964    -0.01706667]
 [-0.01526667 -0.00031     0.00068333 -0.01094333  0.00486667]]
